In [ ]:
import requests
from matplotlib import pyplot as plt

In [ ]:
with open("../apikey.txt", "r") as IN:
    APIKEY = IN.read()

In [ ]:
APIKEY

## Wie viele Objekte je Sektor gibt es?

### Liste der Sektoren

In [ ]:
headers = {
    'Accept' : 'application/json',
    'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
}

res = requests.get('https://api.deutsche-digitale-bibliothek.de/institutions/sectors', headers = headers)

sectorList = [_['value'] for _ in res.json()]

### Zahl der Items je Sektor

In [ ]:
def itemsPerSector(sector):
    res = requests.get("https://api.deutsche-digitale-bibliothek.de/search",
             params = {
                 'query' : '*',
                 'facet' : 'sector_fct',
                 'sector_fct' : sector
             },
            headers = headers
                      )
    return res.json()

In [ ]:
sectorStatistics = { s : int(itemsPerSector(s)['numberOfResults']) for s in sectorList }

In [ ]:
xs,ys = zip(*sectorStatistics.items())
plt.bar(xs, ys)
plt.title("Zahl der Objekte je Sektor")
plt.show()

## Datenabzug nach Suchkriterien

Wie ziehe ich mir alle Datensätze zu Käthe Kollwitz?

In [ ]:
endpunkt = "https://api.deutsche-digitale-bibliothek.de/search"

headers = {
'Accept' : 'application/json',
'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
}

parameters = {
    'facet' : 'affiliate_fct_role',
    'affiliate_fct_role' : 'Käthe Kollwitz'
}

res = requests.get(endpunkt, headers = headers, params = parameters)

In [ ]:
for doc in res.json()['results'][0]['docs']:
    item = requests.get(f'https://api.deutsche-digitale-bibliothek.de/items/{doc["id"]}', headers = headers).json()
    print(item)
    break # höre nach dem ersten Durchlauf auf